In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatLogSlider, fixed
from scipy.stats import norm

def n_pdf(x, mu=0., sigma=1.): # normal pdf
    u = (x - mu) / abs(sigma)
    y = (1 / (np.sqrt(2 * np.pi) * abs(sigma)))
    y *= np.exp(-u * u / 2)   
    return y
        
def ksdensity(data, width=0.3):
    """Returns kernel smoothing function from data points in data"""
    def ksd(x_axis):  
        prob = [n_pdf(x_i, data, width) for x_i in x_axis]
        pdf = [np.average(pr) for pr in prob] # each row is one x value
        return np.array(pdf)
    return ksd

def calc_norm_bin_probabilities(bins):
    '''Returns an array containing the probabilities of a gaussian number lying in each bin'''
    return[norm.cdf(bins[i+1]) - norm.cdf(bins[i]) for i in range(len(bins)-1)]

def calc_norm_scaler(func_width, no_bins, n, func_max):
    '''Returns the scaling factor to scale a standard normal to a histogram 
    by calculating the probability of a random gaussian number lying in 
    a bin centred on zero and therefore the proportion of histogram data
    that you'd expect to lie in this bin'''
    delta = func_width/no_bins
    area = norm.cdf(0+delta/2) - norm.cdf(0-delta/2)
    return area*n/func_max

def calc_scaler(bins, bin_heights, x, y, N, asymptotes=False):
    '''Returns a scaling factor to scale a pdf to a histogram representing
    a probability distribution. Using the Jacobian transformation 
    to generate the pdf of a function of a random variable can lead to 
    asymptotes in the pdf when there are turning points in the function. 
    The  effects of this can be minimised by setting the asymptotes argument 
    to true'''
    
    #Calculate the area covered by the histogram and estimate area under pdf using trapezium rule
    pdf_area = sum(abs(x[i+1] - x[i]) * (y[i+1] + y[i])/2 for i in range(len(x) - 1))
    hist_area = sum((bins[i+1] - bins[i]) * bin_heights[i] for i in range(len(bin_heights)))
    scaler = hist_area/pdf_area
    
    if not asymptotes:
        return scaler
    
    else:
        #A cutoff is used to minimise the affects of super large values near the asymptotes of the pdf
        #During numerical integration, any values greater than the cutoff will be replaced with the cutoff value
        cutoff = 2 * max(bin_heights)/scaler 
        pdf_area_a = sum(abs((x[i+1] - x[i]) * ((min(y[i+1], cutoff) + min(y[i], cutoff))/2)) 
                       for i in range(len(x) - 1))
    
    scaler = hist_area/pdf_area_a
    return scaler

def theoretical_mean_and_std(N, bin_probs, no_bins):
    '''Returns the theoretical mean and standard deviation of the histogram count data'''
    mean = sum(N*prob for prob in bin_probs)/no_bins
    var = sum(N*prob*(1-prob) for prob in bin_probs)/no_bins
    return [mean, np.sqrt(var)]

## Uniform and Normal Random Variables

In [2]:
def plot_smoothed_normal_with_histo(N, width=0.3):
    # Generate and plot histogram of N gaussian random numbers
    fig, ax = plt.subplots(1)
    x = np.random.randn(int(N))
    ax.hist(x, bins=30, color='blue') 

    #Generate smoothed density function and x values for plotting
    ks_density = ksdensity(x, width)
    x_values = np.linspace(-5., 5., 100)
    
    #Calculate Scaling factor
    func_range = max(list(ax.hist(x, bins=30))[1]) - min(list(ax.hist(x, bins=30))[1]) #find range of hist data
    max_ks = max(ks_density(x_values))
    norm_scaler = calc_norm_scaler(func_range, 30, N, max_ks)
    
    #Plot
    ax.plot(x_values, norm_scaler * ks_density(x_values), color='magenta')
    ax.set_title('Histogram of Randomly Generated Gaussian Numbers with Smoothing Function')

def plot_smoothed_uniform_with_histo(N, width=0.3):
    # Generate and plot histogram of 1000 uniform random numbers
    fig2, ax2 = plt.subplots(1)
    x = np.random.rand(int(N))
    ax2.hist(x, bins=30, color='blue')
    
    #Generate smoothed density function and x values for plotting
    ks_density = ksdensity(x, width)
    x_values = np.linspace(-1., 2., 100)
    
    #Calculate Scaling factor
    scaler_u = np.mean(list(ax2.hist(x, bins=30))[0]) / max(ks_density(x_values))
    
    #Plot
    ax2.plot(x_values, scaler_u * ks_density(x_values), color='magenta')
    ax2.set_title('Histogram of Uniform Random Numbers with Smoothing Function')  

interact(plot_smoothed_normal_with_histo, width=(0, 2, 0.1), N=FloatLogSlider(min=1, max=4, step=1))
interact(plot_smoothed_uniform_with_histo, width=(0.01, 0.55, 0.04), N=FloatLogSlider(min=1, max=4, step=1))

interactive(children=(FloatLogSlider(value=10.0, description='N', min=1.0, step=1.0), FloatSlider(value=0.3, d…

interactive(children=(FloatLogSlider(value=10.0, description='N', min=1.0, step=1.0), FloatSlider(value=0.3, d…

<function __main__.plot_smoothed_uniform_with_histo(N, width=0.3)>

In [3]:
def plot_normal_with_histo(N, no_bins, sd_lines = True, mean = True):
    fig_n, ax_n = plt.subplots(1, figsize = (8, 5))
    x_vals_n = np.linspace(-5., 5., 1000)
    rand_n = np.random.randn(int(N)) #uniformly generate N random  gaussian numbers
    
    #Calculate Scaling Factor
    func_range = max(list(ax_n.hist(rand_n, bins=no_bins))[1]) - min(list(ax_n.hist(rand_n, bins=no_bins))[1]) 
    max_norm = max(n_pdf(x_vals_n))
    scaler_n = calc_norm_scaler(func_range, no_bins, N, max_norm) 
    
    #Create lists of bins, their data counts and their associated probabilities
    bin_heights_n = ax_n.hist(rand_n, bins=no_bins)[0]
    bins_n = ax_n.hist(rand_n, bins=no_bins)[1]
    bin_probs_n = calc_norm_bin_probabilities(bins_n)
    
    #Calculate theoretical mean and standard deviation of the histogram count data
    theo_mean, theo_std = theoretical_mean_and_std(N, bin_probs_n, no_bins)
    
    #Plotting
    if mean == True:
        ax_n.hlines(theo_mean, bins_n[0], bins_n[-1], color='magenta', 
                    label='Theoretical mean of Hist Data')
        
    if sd_lines == True:
        ax_n.hlines([theo_mean+3*theo_std, theo_mean-3*theo_std], bins_n[0], bins_n[-1], 
                color='black', label='+/- 3 SD lines')
    
    ax_n.hist(rand_n, bins=no_bins, color='orange', label='Histogram Data')
    ax_n.plot(x_vals_n, scaler_n * n_pdf(x_vals_n), color='lime', label='Ideal Normal Distribution')
    ax_n.set_title('Histogram of Randomly Generated Gaussian Numbers with Normal Distribution')
    ax_n.legend()

    
def plot_uniform_with_histo(N, no_bins, sd_lines = True, mean = True):
    fig_u, ax_u = plt.subplots(1, figsize = (8, 5))
    x_vals_u = np.linspace(-0., 1., 1000)
    rand_u = np.random.rand(int(N)) #uniformly generate N gaussian random numbers
    
    scaler_u = np.mean(list(ax_u.hist(rand_u, bins=no_bins))[0]) #height of ideal uniform distribution is mean bar height
    
    #Create lists of bins, their data counts and their associated probabilities
    bin_heights_u = ax_u.hist(rand_u, bins=no_bins)[0]
    bins_u = ax_u.hist(rand_u, bins=no_bins)[1]
    bin_probs_u = np.ones(no_bins) / no_bins
    
    theo_mean, theo_std = theoretical_mean_and_std(N, bin_probs_u, no_bins)
    
    #Plotting
    if mean == True:
        ax_u.hlines(theo_mean, bins_u[0], bins_u[-1], color='magenta', 
                    label='Theoretical mean of Hist Data')
        
    if sd_lines == True:
        ax_u.hlines([theo_mean+3*theo_std, theo_mean-3*theo_std], bins_u[0], bins_u[-1], 
                color='black', label='+/- 3 SD lines')
        
    ax_u.hist(rand_u, bins=no_bins, color='orange', label='Histogram Data')
    ax_u.plot(x_vals_u, scaler_u * np.ones(len(x_vals_u)), color='lime', label='Ideal Uniform Distribution')
    ax_u.set_title('Histogram of Uniform Random Numbers with Uniform Distribution')
    ax_u.legend()
    
interact(plot_normal_with_histo, N=FloatLogSlider(min=1, max=4, step=1), no_bins=(5, 100, 5))
interact(plot_uniform_with_histo, N=FloatLogSlider(min=1, max=4, step=1), no_bins=(5, 100, 5))

interactive(children=(FloatLogSlider(value=10.0, description='N', min=1.0, step=1.0), IntSlider(value=50, desc…

interactive(children=(FloatLogSlider(value=10.0, description='N', min=1.0, step=1.0), IntSlider(value=50, desc…

<function __main__.plot_uniform_with_histo(N, no_bins, sd_lines=True, mean=True)>

## Functions of Random Variables

In [4]:
def y1(x, a=3, b=5, diff=False, inverse=False):
    '''Returns function, its derivative or inverse'''
    if diff == True:
        return a
    elif inverse == True:
        return [(x - b)/a]
    else:
        return a*x + b

    
def y2(x, diff=False, inverse=False):
    '''Returns function, derivative or inverse'''
    if diff == True:
        return 2*x
    elif inverse == True:
        return [np.sqrt(x), -np.sqrt(x)]
    else:
        return x**2


def plot_func(N, title, f, no_bins, pdf, axes_lim=False, trig=False, asym=False,
              dist_type='gaussian'):
    if trig:
        rand_x = np.random.uniform(0, 2*np.pi, int(N))
    else:
        rand_x = np.random.randn(int(N))
    
    #Generate ordered (needed for numerical integration) list of evenly spaced y values for a given x range
    x_vals_hist = np.linspace(-5., 5., 1000)
    y_vals_hist_unsorted = f(x_vals_hist) #for non 1-1 functions this list will not be ordered
    start =  min(y_vals_hist_unsorted)
    stop = max(y_vals_hist_unsorted)
    y_vals_hist = np.linspace(start, stop, 1000)
    
    
    fig, ax = plt.subplots(2, figsize = (10, 10))
    
    #Generate p(y) using the Jacobian Transformation
    prob_y = np.array([(sum(pdf(x_k)/abs(f(x_k, diff=True)) for x_k in f(y, inverse=True))) 
                        for y in y_vals_hist])
    
    #Create lists of bin edges and bin heights
    bins = list(ax[1].hist(f(rand_x), bins=no_bins)[1])
    bin_heights = list(ax[1].hist(f(rand_x), bins=no_bins)[0])    
    
    #Calculate scaling factor to scale pdf to histogram
    scaler = calc_scaler(bins, bin_heights, y_vals_hist, prob_y, N, asymptotes=asym)   
    
    #Plot f(x) as a function of random variable X to visualise distribution
    ax[0].plot(rand_x, f(rand_x), 'x')
    ax[0].set_title(f'f(x) = {title} for random {dist_type} variable X')
    ax[0].set_xlabel('x')
    ax[0].set_ylabel('f(x)')
    ax[0].grid()
    
    #Limits the size of the axes to the histogram data rather the pdf plot
    if axes_lim:
        ax[1].set_ylim(bottom=0, top=1.2*max(list(ax[1].hist(f(rand_x), bins=no_bins)[0])))
        ax[1].set_xlim(left= 1.1*min(list(ax[1].hist(f(rand_x), bins=no_bins)[1])), 
                       right=1.1*max(list(ax[1].hist(f(rand_x), bins=no_bins)[1])))
    
    #Plot histogram and overlay pdf
    ax[1].hist(f(rand_x), bins=no_bins, color='violet', label='Histogram Data')
    ax[1].plot(y_vals_hist, prob_y*scaler, color='black', label='PDF')
    ax[1].set_title(f'Probability distribution of {title} for random {dist_type} variable X')
    ax[1].set_xlabel(f'{title}')
    ax[1].set_ylabel(f'P({title})')
    ax[1].legend()
    
       
interact(plot_func, N=FloatLogSlider(min=1, max=4, step=1), title=fixed('3x + 5'), f=fixed(y1), 
         no_bins=(5, 50, 5), pdf=fixed(n_pdf), axes_lim=fixed(False), trig=fixed(False),
        asym=fixed(False), dist_type=fixed('gaussian'))

interact(plot_func, N=FloatLogSlider(min=1, max=4, step=1), title=fixed('x^2'), f=fixed(y2), 
         no_bins=(5, 50, 5), pdf=fixed(n_pdf), axes_lim=fixed(True), trig=fixed(False),
        asym=fixed(True), dist_type=fixed('gaussian'))

interactive(children=(FloatLogSlider(value=10.0, description='N', min=1.0, step=1.0), IntSlider(value=25, desc…

interactive(children=(FloatLogSlider(value=10.0, description='N', min=1.0, step=1.0), IntSlider(value=25, desc…

<function __main__.plot_func(N, title, f, no_bins, pdf, axes_lim=False, trig=False, asym=False, dist_type='gaussian')>

In [5]:
def y4(x, diff=False, inverse=False):
    '''Returns function, derivative or inverse'''
    if diff == True:
        return np.cos(x)
    elif inverse == True:
        if np.arcsin(x) > 0:
            return[np.arcsin(x), np.pi - np.arcsin(x)]
        else:
            return[2*np.pi + np.arcsin(x), np.pi - np.arcsin(x)]
    else:
        return np.sin(x)
    
def uniform_trig_pdf(x):
    if 0 <= x <= 2*np.pi:
        return 1/(2*np.pi)
    else:
        return 0


interact(plot_func, N=FloatLogSlider(min=1, max=4, step=1), title=fixed('sin(x)'), f=fixed(y4), 
         no_bins=(5, 50, 5), pdf=fixed(uniform_trig_pdf), axes_lim=fixed(True), trig=fixed(True),
        asym=fixed(True), dist_type=fixed('uniform'))

interactive(children=(FloatLogSlider(value=10.0, description='N', min=1.0, step=1.0), IntSlider(value=25, desc…

<function __main__.plot_func(N, title, f, no_bins, pdf, axes_lim=False, trig=False, asym=False, dist_type='gaussian')>

## Inverse CDF Method

In [6]:
def distribution1(y, pdf=True, cdf=False, inv_cdf=False):
    if cdf:
        return 1 - np.exp(-y)
    elif inv_cdf:
        return -np.log(1-y)
    else:
        return np.exp(-y)

def plot_using_inverse_CDF(N, no_bins, dist, kernel_width, title, show_kd_estimate=True):
    rand_x = np.random.rand(int(N))
    y_vals = np.linspace(0., 10., 1000)
    
    fig, ax = plt.subplots(1, figsize = (10, 10))
    
    Y = dist(rand_x, inv_cdf=True)
    prob_y = dist(y_vals)
    ks_density = ksdensity(Y, width=kernel_width)
    smoothed = ks_density(y_vals)
    
    bins = list(ax.hist(Y, bins=no_bins)[1])
    bin_heights = list(ax.hist(Y, bins=no_bins)[0])
    
    scaler_ks = calc_scaler(bins, bin_heights, y_vals, smoothed, N)
    scaler_pdf = calc_scaler(bins, bin_heights, y_vals, prob_y, N)
    
    ax.hist(Y, bins=no_bins, color='violet', label='Histogram Data')
    if show_kd_estimate:
        ax.plot(y_vals, smoothed*scaler_ks, color='black', label='Kernel Density Estimate')
    ax.plot(y_vals, prob_y*scaler_pdf, color='green', label=f'p(y) = {title}')
    ax.legend()

    
interact(plot_using_inverse_CDF, N=FloatLogSlider(min=1, max=4, step=1), dist=fixed(distribution1), 
         no_bins=(5, 50, 5), kernel_width=(0.05, 1, 0.05), title=fixed('e^-y'))

interactive(children=(FloatLogSlider(value=10.0, description='N', min=1.0, step=1.0), IntSlider(value=25, desc…

<function __main__.plot_using_inverse_CDF(N, no_bins, dist, kernel_width, title, show_kd_estimate=True)>

## Simulation from Non-Standard Densities

In [7]:
def U_dist(u, a, inv_cdf=False):
    if inv_cdf:
        return -2 * np.log(1-u) / a**2
    else:
        return a**2/2 * np.exp(-a**2 * u/2)

def plotX(a, N, no_bins, width):
    x = np.linspace(-5.,  5., 1000)
    rand_unif = np.random.rand(int(N)) #generate uniformally distributed random variable
    fig, ax = plt.subplots(3, figsize = (10, 10))
    
    #generate distribution for U using inverse cdf and uniform random variable
    U = U_dist(rand_unif, a, inv_cdf=True)
    
    #generate random variable X from gaussian with variance U 
    X = [np.random.normal(loc=0, scale=np.sqrt(U_i)) for U_i in U]
    
    ks_density = ksdensity(X, width=width)
    
    ax[0].hist(U, bins=no_bins, color='violet')
    ax[0].set_title('Distribution of random variable U')
    ax[1].hist(X, bins=no_bins, color='violet')
    ax[1].set_title('Distribution of random variable X taken from normal with variance U')
    
    ax[2].plot(x, np.log(ks_density(x)), color = 'orange', label = f'alpha = {a}')
    ax[2].set_title('Natural Log of Kernel Density Estimate')
    ax[2].grid()
    
    

interact(plotX, a = (0.1, 10, 1), N=FloatLogSlider(min=1, max=4, step=1), no_bins=(5, 50, 5), 
         width=(0, 5, 0.5))

interactive(children=(FloatSlider(value=4.1, description='a', max=10.0, min=0.1, step=1.0), FloatLogSlider(val…

<function __main__.plotX(a, N, no_bins, width)>